In [33]:
import requests
import pandas as pd
import numpy as np
import json


In [34]:
data = pd.read_csv("./data/Obesity Classification.csv")


In [35]:
data

,ID,Age,Gender,Height,Weight,BMI,Label
0,1,25,Male,175,80,25.3,Normal Weight
1,2,30,Female,160,60,22.5,Normal Weight
2,3,35,Male,180,90,27.3,Overweight
3,4,40,Female,150,50,20.0,Underweight
4,5,45,Male,190,100,31.2,Obese
...,...,...,...,...,...,...,...
103,106,11,Male,175,10,3.9,Underweight
104,107,16,Female,160,10,3.9,Underweight
105,108,21,Male,180,15,5.6,Underweight
106,109,26,Female,150,15,5.6,Underweight


In [36]:
data["Label"].unique()

array(['Normal Weight', 'Overweight', 'Underweight', 'Obese'],
      dtype=object)

In [37]:
print(requests.get("https://obesityclassification-production.up.railway.app/v1/models/cc-model").json())


{'model_version_status': [{'version': '1751687975', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}]}


In [38]:

def prepare_json(instance: dict):
    # Pastikan semua NumPy array dikonversi ke list
    for k, v in instance.items():
        if isinstance(v, np.ndarray):
            instance[k] = v.tolist()
    payload = {
        "signature_name": "serving_default",
        "instances": [instance]
    }
    return json.dumps(payload)

instance = {
    "inputs": [[170.0]],    # Height
    "inputs_1": [[70.0]],   # Weight
    "inputs_2": [[0]],      # Gender 
    "inputs_3": [[25.0]],   # BMI
    "inputs_4": [[30.0]]      # Age
}

response = requests.post(
    "https://obesityclassification-production.up.railway.app/v1/models/cc-model:predict",
    headers={"Content-Type": "application/json"},
    data=prepare_json(instance),
)

print(response.status_code)
print(response.json())


400
{'error': '{{function_node __inference_pruned_247885_specialized_for_StatefulPartitionedCall_2_StatefulPartitionedCall_transform_features_layer_StatefulPartitionedCall_at_tf_graph_specialized_for_StatefulPartitionedCall_2_StatefulPartitionedCall_transform_features_layer_StatefulPartitionedCall_at_tf_graph}} {{function_node __inference_pruned_247885_specialized_for_StatefulPartitionedCall_2_StatefulPartitionedCall_transform_features_layer_StatefulPartitionedCall_at_tf_graph_specialized_for_StatefulPartitionedCall_2_StatefulPartitionedCall_transform_features_layer_StatefulPartitionedCall_at_tf_graph}} Expects arg[0] to be int64 but float is provided\n\t [[{{function_node __inference_serve_json_fn_248130}}{{node transform_features_layer/StatefulPartitionedCall}}]]'}
